# Data Dive 2: Loading and Summarizing Data
### Part 2: Scraping the Web for Unique Data

#### [Web scraping](https://en.wikipedia.org/wiki/Web_scraping) is the process of extracting data from html code on the internet. 

Resources on web scraping:
* [Digital Ocean Tutorial](https://www.digitalocean.com/community/tutorials/how-to-scrape-web-pages-with-beautiful-soup-and-python-3) (requests, Beautiful Soup)
* [DataCamp Tutorial](https://www.datacamp.com/community/tutorials/web-scraping-using-python) (urllib, Beautiful Soup)
* [Hitchhiker's Guide to Python](https://docs.python-guide.org/scenarios/scrape/) (requests, lxml) 

**Important Note**: This is for demonstration purposes only, and only harvests content from individual pages. When building a scraper to harvest large amounts of data from multiple pages, be mindful of [legal](https://www.fastcompany.com/40456140/bots-are-scraping-your-public-data-for-cash-amid-murky-laws-and-ethics-linkedin-hiq) and [ethical](https://towardsdatascience.com/ethics-in-web-scraping-b96b18136f01) issues in web scraping.  

## Today's Exercise
Say we want to learn more about where Google's offices are located. Helpfully, the provide a list of all of their campuses globally at [google.com/about/locations](https://www.google.com/about/locations). However, copying this list by hand to do data analysis on would be frustrating and time-consuming. Let's take a look at how web scraping can make this process easier. 

First, let's import all of the packages we'll need for today's exercise. There are a wide variety of packages that can be helpful, but today we'll be using *requests* and *Beautiful Soup* to pull the contents of these websites. 

In [1]:
import re
import pandas as pd
from bs4 import BeautifulSoup as soup
import requests


First, we use the *requests* package to get the content of the google site we'd like to extract office location information from: 

In [70]:
url = 'https://www.google.com/about/locations/'
site_source = requests.get(url)

This is going to give us an enormous amount of content - everything we would get if we looked directly at the source code in the browser. 

In [71]:
print(site_source.text)

<!DOCTYPE html>










<html lang="en" dir="ltr" class="google spa"  locale="en_us">
  <head>

        <meta charset="utf-8">

        <meta name="viewport" content="initial-scale=1, minimum-scale=1, width=device-width">

        <title>Our Locations | Google</title>

        <meta name="description" content="Google has more than 70 offices in 50 countries. View a directory of our locations around the world.">

        <link href="//fonts.googleapis.com/css?family=Roboto:100,300,400,500,700&lang=en" rel="stylesheet">
        <link href="//fonts.googleapis.com/css?family=Google+Sans:400,500,700,900|Google+Sans+Display:400,500" rel="stylesheet">

        <link href="/about/assets/css/main.min.css?cache=e2cb111" rel="stylesheet">



      <meta property="og:description" content="Google has more than 70 offices in 50 countries. View a directory of our locations around the world.">
      <meta property="og:title" content="Our Locations | Google">
      <meta property="og:image" content="

#### We could parse this ourselves, but fortunately scraping packages make this much easier

We'll use Beautiful Soup's built in functionality to extract info on the individual offices.

First, we parse the full site content.

In [84]:
site_content = soup(site_source.content, "html.parser")

type(site_content)

bs4.BeautifulSoup

Next, we pick out the office elements

In [86]:
offices = site_content.select(".office-info")

len(offices)

156

Now that we've isolated the office elements, let's extract the location name and address for each.

In [88]:
for o in offices:
    office = o.select(".office-name")[0].string.strip()
    address = o.select(".office-address")[0].string.strip()
    
    print(office)
    print(address)
    print()

Dubai
TECOM Zone, Dubai Internet City
Dubai, United Arab Emirates

Haifa
Building 30
MATAM, Advanced Technology Center
Haifa, 3190500
Israel

Istanbul
Eski Buyukdere Caddesi No: 209
34394
Istanbul, Turkey

Johannesburg
35 Ballyclare Drive Building E
Johannesburg
2191, South Africa

Tel Aviv
Yigal Alon 98
Tel Aviv, 6789141
Israel

Bangalore
No. 3, RMZ Infinity - Tower E
Old Madras Road
4th and 5th Floors
Bangalore, 560 016, India

Bangkok
57 Park Ventures Ecoplex
14th Floor, Wireless Road
Bangkok, 10330, Thailand

Beijing
6th Floor, Tower B, Raycom InfoTech Park 
No. 2 Kexueyuan South Road 
Zhongguancun Beijing 100190

Guangzhou
Taikoo Hui Tower 1, No.383 Tianhe Road
Guangzhou, 510620
China

Gurgaon
Sector 15, Part II Village Silokhera
Gurgaon 122001
India

Hong Kong
1 Matheson Street
Causeway Bay, Hong Kong

Hyderabad
Survey No. 13, DivyaSree Omega
Kondapur Village
Hyderabad, Telangana 500084
India

Jakarta
Jalan Asia Afrika No. 8, Sentral Senayan 2 Lt. 28
Kebayoran Baru, Jakarta 10270

#### If we look carefully at our extracted elements, we'll see we have some issues:
1. All elements appear twice.
2. The zip codes - which we're interested in - are part of broader strings. 

These are trivial to handle, we'll just need to pass over the data carefully to handle both. 

In [95]:
us_offices = []
for o in offices:
    office = o.select(".office-name")[0].string.strip()
    address = o.select(".office-address")[0].string.strip()

    is_US = re.search(r'(United States)', address)

    if is_US:
        
        print(office)
        zip_code = re.search(r'(\d{5})', address)
        if zip_code:
            print(zip_code.group())
            if [office, zip_code.group()] not in us_offices:
                us_offices.append([office, zip_code.group()])
        print()

Ann Arbor
48105

Atlanta
30309

Austin
78701

Birmingham
48009

Boulder
80302

Cambridge
02142

Chapel Hill
27516

Chicago
60607

Irvine
19510

Kirkland

Los Angeles
90291

Miami
33131

Mountain View
94043

New York
10011

Pittsburgh
15206

Playa Vista
12422

Reston
20190

San Bruno
94066

San Diego
92121

San Francisco
94105

Seattle
98103

Sunnyvale
94089

Washington DC
20001

Ann Arbor
48105

Atlanta
30309

Austin
78701

Birmingham
48009

Boulder
80302

Cambridge
02142

Chapel Hill
27516

Chicago
60607

Irvine
19510

Kirkland

Los Angeles
90291

Miami
33131

Mountain View
94043

New York
10011

Pittsburgh
15206

Playa Vista
12422

Reston
20190

San Bruno
94066

San Diego
92121

San Francisco
94105

Seattle
98103

Sunnyvale
94089

Washington DC
20001



Now that we've extracted a list of offices and zip codes, we can load them into a data frame.

In [97]:
office_df = pd.DataFrame(us_offices, columns=['Office', 'Zip Code'])

office_df.head(5)

,Office,Zip Code
0,Ann Arbor,48105
1,Atlanta,30309
2,Austin,78701
3,Birmingham,48009
4,Boulder,80302


## Exercise: In What Countries Does Google Maintain Offices?

In [107]:
# todo: compile a dataframe the cities and countries in which Google maintains international offices

## Adding County Information

The Department of Housing and Urban Development makes a *crosswalk* of zip codes to counties available [here](https://www.huduser.gov/portal/datasets/usps_crosswalk.html). We can load these into pandas and clean them up to find the county for each office. 

In [98]:
zip_df = pd.read_excel('https://grantmlong.com/data/ZIP_COUNTY_122016.xlsx')

zip_df.head(5)

,ZIP,COUNTY,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO
0,501,36103,0.000000,1.000000,0.000000,1.000000
1,601,72001,1.000000,1.000000,1.000000,1.000000
2,602,72003,1.000000,1.000000,1.000000,1.000000
3,603,72071,0.008258,0.000948,0.007767,0.007841
4,603,72005,0.991742,0.999052,0.992233,0.992159


#### A good rule of thumb: if two numbers cannot be added together to produce a logical result, they should be stored as strings. '

We can recast the zip and county ids as strings - with leading zeros - to make this dataframe easier to handle. To do this we can use the [.astype()]() and [.zfill()]() methods.

In [99]:
zip_df['Zip Code'] = zip_df['ZIP'].astype(str).str.zfill(5) 
zip_df['County Number'] = zip_df['COUNTY'].astype(str).str.zfill(5) 

zip_df.sort_values(by='COUNTY').head(5)

,ZIP,COUNTY,RES_RATIO,BUS_RATIO,OTH_RATIO,TOT_RATIO,Zip Code,County Number
18372,36758,1001,0.200739,0.083333,0.000000,0.197133,36758,01001
18029,36091,1001,0.096154,0.076923,0.000000,0.095303,36091,01001
18008,36068,1001,1.000000,1.000000,1.000000,1.000000,36068,01001
18006,36067,1001,1.000000,0.997963,1.000000,0.999856,36067,01001
18005,36066,1001,0.879960,0.703650,0.677686,0.860627,36066,01001


Of course we don't need all of these columns, but we do need to attach the ***County Number*** column to our Google office data in order to learn more about the data. The [.merge()]() method allows us to do this easily in one line. 

In [100]:
office_df = office_df.merge(zip_df[['Zip Code', 'County Number']], 
                            how='left')

office_df.sort_values(by='Office')
office_df.shape

(23, 3)

In [101]:
office_df.sort_values(by='County Number')


,Office,Zip Code,County Number
10,Los Angeles,90291,06037
18,San Diego,92121,06073
19,San Francisco,94105,06075
17,San Bruno,94066,06081
12,Mountain View,94043,06085
21,Sunnyvale,94089,06085
4,Boulder,80302,08013
22,Washington DC,20001,11001
11,Miami,33131,12086
1,Atlanta,30309,13121


### Merge Office Data with Census Data

In [102]:
census_df = pd.read_csv('https://grantmlong.com/data/census_counties.csv')
census_df['County Number'] = census_df['County Number'].astype(str).str.zfill(5) 

census_df.head(5)

,County Name,Total Population,Median Household Income,County Number
0,"Autauga County, Alabama",55049,53099,01001
1,"Baldwin County, Alabama",199510,51365,01003
2,"Barbour County, Alabama",26614,33956,01005
3,"Bibb County, Alabama",22572,39776,01007
4,"Blount County, Alabama",57704,46212,01009


In [103]:
full_df = census_df.merge(office_df, how='left')
full_df.loc[full_df['Office'].notnull(), ].head(5)

,County Name,Total Population,Median Household Income,County Number,Office,Zip Code
204,"Los Angeles County, California",10057155,57952,06037,Los Angeles,90291
222,"San Diego County, California",3253356,66529,06073,San Diego,92121
223,"San Francisco County, California",850282,87701,06075,San Francisco,94105
226,"San Mateo County, California",754748,98546,06081,San Bruno,94066
228,"Santa Clara County, California",1885056,101173,06085,Mountain View,94043


In [104]:
print(full_df['Median Household Income'].mean())
print(full_df.loc[full_df['Office'].notnull(), 'Median Household Income'].mean())
print(full_df.loc[full_df['Office'].isnull(), 'Median Household Income'].mean())

47268.769326296184
71948.17391304347
47091.27517198249


In [108]:
(full_df
 .sort_values(by='Median Household Income', 
              ascending=False)
 .head(20))

,County Name,Total Population,Median Household Income,County Number,Office,Zip Code
2873,"Loudoun County, Virginia",362435,125672,51107,NaN,NaN
2926,"Falls Church city, Virginia",13597,115244,51610,NaN,NaN
2849,"Fairfax County, Virginia",1132887,114329,51059,Reston,20190
1206,"Howard County, Maryland",308447,113800,24027,NaN,NaN
2827,"Arlington County, Virginia",226092,108706,51013,NaN,NaN
1784,"Hunterdon County, New Jersey",125708,108177,34019,NaN,NaN
1811,"Los Alamos County, New Mexico",17895,105902,35028,NaN,NaN
263,"Douglas County, Colorado",314238,105759,08035,NaN,NaN
2925,"Fairfax city, Virginia",23620,104065,51600,NaN,NaN
1788,"Morris County, New Jersey",498215,102798,34027,NaN,NaN


## Exercise: What Other Interesting Findings Can We Identify?